# Retrieve RD FASTA sequences
#### DATE: 07-08-19
#### TASK: 
####       - Retrive coordinates of RD from the RD.csv file
####       - Use coords to extracts RD in FASTA from the H37rv reference genome
####       - Create blast db from RDs
####       - Blast mbovis/morygis against local db

In [144]:
import os, sys, io, random, subprocess
import string
import numpy as np
import pandas as pd
pd.set_option('display.width',150)
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

sys.path.append('/home/tortoise/pybioviz/')
from pybioviz import viewers, utils

### The H37rv genome is going to be converted to a string, to be sliced

In [145]:
# Here we open up the H37rv genome.
# The RD are defined by coordinates on this genome so we will use the RD sequences spliced from this
def extract_genome(genomefile):
    
    # This string will have each line of genomic sequence added to it
    genome_string = ''
    
    # The genome file is opened here
    with open(genomefile) as gfile:
    
        # Each line is read
        for line in gfile:
    
            # We skip the fasta header
            if '>' in line:
                continue
            # The genome string is then assembled...
            else:
                genome_string = genome_string + line
    # and returned    
    return genome_string

# The retrieval of the genome is called
MTBenome = extract_genome('ref_genomes/MTB_h37rv.fna')

### The .csv file is going to be opened, with relevent data extracted and stored

In [156]:
# This function reads the csv and takes needed information
# It stores the uses an inputted dictionary
# The key added is each RD name
# The value added is the slice of the H37rv genome using the csv Start and Stop coordinates as slice coordinates
# RDict is the empty dictionary used to store info
def extract_csv(csv_file, RDict, RDcount_list):
    
    # RD is the dataframe made from the .csv file
    # Commented .csv inputs are ignored
    RD = pd.read_csv(csv_file)#, comment='#')
    
    # Each row is iterated over using iterrows
    # This creates a set of each row, where each value can be called using the column name
    for iteration, row in RD.iterrows():
        
        # RDict takes in the row's RD name as a key..
        # .. and a genome slice as a value
        # The slice is specified using the RD's Start and Stop coordinates
        RDict[row.RD_name] = MTBenome[row.Start:row.Stop+1]
        for RDcount in RDcount_list:
            RDcount[row.RD_name] = 0

# This empty dict later cathes .csv information of each RD
RDict = {}

# These dict is going to be used to count results for each RD
mbo_RDcount = {}
mor_RDcount = {}
mca_RDcount = {}
mmi_RDcount = {}

# Here a list of dicts is being defined to be parsed through the csv extraction function
RDcount_list = [mbo_RDcount, mor_RDcount, mca_RDcount, mmi_RDcount]

# The .csv file is parsed through our extraction function
extract_csv('RD.csv', RDict, RDcount_list)

In [147]:
# Here the extracted files are written in fasta format to an output file for later blasting
# A new file is created
with open('RD_seq.fasta', 'w+') as fna:
    
    # We iterate over the RD dictionary's items
    for RD, seq in RDict.items():
        
        # And write into the output file each RD and its sequence..
        # .. in fasta format
        # FASTA format is needed for blasting
        fna.write(f'>{RD}\n{seq}\n')


In [157]:
# Here we create a database of the RDs, which can align each genome against
# The function takes in the created RD fasta file
def makedb(RDfna):
    
    # a command is then created using the inputted file
    # The inputted file is the database to be created, using the -in parameter
    # -dtype is nucl as we are creating a nucleotide database
    # The database will then be called the same as the input file
    make_db = f'makeblastdb -in {RDfna} -dbtype nucl'
    
    # The command is used on the command line
    os.system(make_db)
    
    # A return could be used here to pass on the name of the database created..
    
# The database is created using our created file    
makedb('RD_seq.fasta')

In [158]:
# Here we perform the blast search with our db
def do_blast(genome):
    
    # This is the created output file name, made frpom whichever genome is being parsed
    outfile = os.path.basename(genome).split('.')[0] + 'blast.txt'
    
    # The cmd is then:
    
    # - created
    blastcmd = f'blastn -query {genome} -db RD_seq.fasta -out {outfile} -outfmt 7'
    
    # - printed
    print(blastcmd)
    
    # - called on the cmd line
    os.system(blastcmd)
    
    # the output file name is returned, and stored as a variable
    return outfile

# the blastin function is called for each genome, and the name of the output file is stored
mbo_blast = do_blast('ref_genomes/mbovis.fna')
mor_blast = do_blast('ref_genomes/morygis_ncbi.fna')
mca_blast = do_blast('ref_genomes/mcap.fna')
mmi_blast = do_blast('ref_genomes/mmic.fna')

blastn -query ref_genomes/mbovis.fna -db RD_seq.fasta -out mbovisblast.txt -outfmt 7
blastn -query ref_genomes/morygis_ncbi.fna -db RD_seq.fasta -out morygis_ncbiblast.txt -outfmt 7
blastn -query ref_genomes/mcap.fna -db RD_seq.fasta -out mcapblast.txt -outfmt 7
blastn -query ref_genomes/mmic.fna -db RD_seq.fasta -out mmicblast.txt -outfmt 7


In [159]:
## We then read the blast results, taking in the blast file
def read_blast_res(blast_res, RDcount):
    # This is just a simple count of how many hits total came up
    RD_hit_count = 0
    
    # The results file is opened
    with open(blast_res) as res:
        
        # Each line is read
        for line in res:
            
            #ignoring commented lines..
            if line.startswith('#'):
                continue
            
            # ignoring low identity hits..
            elif float(line.split('\t')[2]) <= 95 or float(line.split('\t')[3]) <= 200:
                continue
            
            
            # Each result is printed/counted
            else:
                #print(line)
                RD_hit_count += 1
                RDcount[line.split('\t')[1]] += 1
    
    # The hit count is then returned
    return RD_hit_count

# Each blast count is called..
mbo_RD_hits = read_blast_res(mbo_blast, mbo_RDcount)
mor_RD_hits = read_blast_res(mor_blast, mor_RDcount)
mca_RD_hits = read_blast_res(mca_blast, mca_RDcount)
mmi_RD_hits = read_blast_res(mmi_blast, mmi_RDcount)

# ..and hit count is printed
print(f'mbovis hit count: {mbo_RD_hits}')
print(f'morygis hit count: {mor_RD_hits}')
print(f'mcap hit count: {mca_RD_hits}')
print(f'mmic hit count: {mmi_RD_hits}')

## Beware that jupyter lab does not reset each dictionary, hence results are appended. Reset in previous tab.

mbovis hit count: 50
morygis hit count: 57
mcap hit count: 52
mmic hit count: 62


In [160]:
print(mbo_RDcount)
print(mor_RDcount)
print(mca_RDcount)
print(mmi_RDcount)

{'#RD9': 1, 'RD711': 1, 'RD702': 1, '#RD4': 1, '#RD1bcg': 1, '#RD1mic': 1, '#RD2seal': 1, '#RD2bcg': 2, '#RD7': 1, '#RD8': 1, '#RD10': 1, '#RD12bov': 1, '#RD12can': 1, 'RD105': 1, 'RD239': 1, 'RD750': 1, 'RD142': 1, 'RD150': 1, 'RD181': 1, 'RD207': 1, 'RD115': 1, 'RD122': 1, 'RD174': 0, 'RD182': 1, 'RD183': 1, 'RD193': 1, 'RD219': 1, 'RD724': 1, 'RD726': 1, 'RD761': 1, '7bp pks15/1': 0, 'RD11': 1, 'RD1': 1, 'RD2': 2, 'RD3': 2, 'RD4': 1, 'RD5': 1, 'RD6': 3, 'RD7': 1, 'RD8': 1, 'RD9': 1, 'RD10': 1, 'RD12': 1, 'RD13': 1, 'RD14': 2, 'RD10_manual': 1}
{'#RD9': 3, 'RD711': 1, 'RD702': 1, '#RD4': 1, '#RD1bcg': 1, '#RD1mic': 1, '#RD2seal': 1, '#RD2bcg': 3, '#RD7': 1, '#RD8': 1, '#RD10': 1, '#RD12bov': 1, '#RD12can': 1, 'RD105': 1, 'RD239': 1, 'RD750': 1, 'RD142': 1, 'RD150': 1, 'RD181': 1, 'RD207': 1, 'RD115': 1, 'RD122': 1, 'RD174': 0, 'RD182': 1, 'RD183': 1, 'RD193': 1, 'RD219': 1, 'RD724': 1, 'RD726': 1, 'RD761': 1, '7bp pks15/1': 0, 'RD11': 1, 'RD1': 1, 'RD2': 3, 'RD3': 2, 'RD4': 1, 'RD5':

In [161]:
RD_hits_df = pd.DataFrame([mbo_RDcount, mca_RDcount, mmi_RDcount, mor_RDcount])
RD_hits_df.index=['mbovis', 'mcaprae', 'mmicroti', 'morygis']
RD_hits_df.T.sort_index()

,mbovis,mcaprae,mmicroti,morygis
#RD10,1,1,1,1
#RD12bov,1,1,1,1
#RD12can,1,1,3,1
#RD1bcg,1,1,1,1
#RD1mic,1,1,1,1
#RD2bcg,2,2,3,3
#RD2seal,1,1,1,1
#RD4,1,1,2,1
#RD7,1,1,1,1
#RD8,1,1,3,1
